In [1]:
import os

os.chdir('/Documents/CBA_Stats')
from sklearn.linear_model import LinearRegression

import pandas as pd
import re
from cbastats import DBHelper
import datetime
import pytz
import numpy as np
# from cbastats import Scraper
from cbastats import ScraperMongo
from cbastats.Team import *
from cbastats.Player import *

# Connect Database

In [2]:
mongodbio =DBHelper.MongoDBHelper() 
client = mongodbio.create_connection(DBHelper.MONGODB_USERNAME,DBHelper.MONGODB_PWD,DBHelper.MONGODB_ENDPOINT)

db = client['cbaStats']

existing database ['cbaStats', 'admin', 'local']


In [3]:
db.list_collection_names()

['cbaAllGamesBoxScore',
 'cbaGamesStaging',
 'uniqueBoxScores',
 'seasonTeamStats',
 'seasonPlayerStats',
 'cbaGames']

In [4]:
coll_games = db['cbaGames']

# Pull Data

In [5]:
all_games = pd.DataFrame(mongodbio.select_records(coll_games,filter={'赛季':'20-21','比分':{"$ne":'VS'}},field={'轮次': 1,
                                            '日期': 1,
                                            '主队': 1,
                                            '比分': 1,
                                            '客队': 1,'_id':0}))

In [6]:
all_games.columns

Index(['轮次', '日期', '主队', '比分', '客队'], dtype='object')

In [7]:
all_games.drop_duplicates(inplace=True)

In [8]:
all_games

,轮次,日期,主队,比分,客队
0,第3轮,2020-10-22 04:30:00,浙江,105:93,山西
1,第4轮,2020-10-24 12:00:00,上海,99:102,北京
2,第5轮,2020-10-28 03:00:00,青岛,113:114,江苏
3,第7轮,2020-11-01 03:00:00,广州,110:111,同曦
4,第8轮,2020-11-04 11:35:00,吉林,105:84,青岛
...,...,...,...,...,...
268,第21轮,2020-12-21 04:30:00,天津,91:105,新疆
269,第21轮,2020-12-21 07:30:00,北控,112:108,吉林
270,第22轮,2020-12-24 04:00:00,浙江,94:106,山西
271,第23轮,2020-12-26 11:35:00,辽宁,125:110,山西


# Processing

In [9]:
all_games[['主队比分','客队比分']]=all_games['比分'].str.split(':',expand=True)
all_games

,轮次,日期,主队,比分,客队,主队比分,客队比分
0,第3轮,2020-10-22 04:30:00,浙江,105:93,山西,105,93
1,第4轮,2020-10-24 12:00:00,上海,99:102,北京,99,102
2,第5轮,2020-10-28 03:00:00,青岛,113:114,江苏,113,114
3,第7轮,2020-11-01 03:00:00,广州,110:111,同曦,110,111
4,第8轮,2020-11-04 11:35:00,吉林,105:84,青岛,105,84
...,...,...,...,...,...,...,...
268,第21轮,2020-12-21 04:30:00,天津,91:105,新疆,91,105
269,第21轮,2020-12-21 07:30:00,北控,112:108,吉林,112,108
270,第22轮,2020-12-24 04:00:00,浙江,94:106,山西,94,106
271,第23轮,2020-12-26 11:35:00,辽宁,125:110,山西,125,110


In [10]:
all_games['客队比分'] = pd.to_numeric(all_games['客队比分'])
all_games['主队比分'] = pd.to_numeric(all_games['主队比分'])

In [11]:
all_games.loc[all_games['主队比分']>all_games['客队比分'],'负方']=all_games['客队']
all_games.loc[all_games['主队比分']<all_games['客队比分'],'负方']=all_games['主队']
all_games.loc[all_games['主队比分']>all_games['客队比分'],'胜方']=all_games['主队']
all_games.loc[all_games['主队比分']<all_games['客队比分'],'胜方']=all_games['客队']

In [12]:
hash_t = {}
num=0
for team in list(all_games['主队'].value_counts().index):
    hash_t[team]=num
    num+=1

In [13]:
hash_t

{'北控': 0,
 '深圳': 1,
 '福建': 2,
 '同曦': 3,
 '广州': 4,
 '吉林': 5,
 '北京': 6,
 '辽宁': 7,
 '新疆': 8,
 '天津': 9,
 '广厦': 10,
 '青岛': 11,
 '山东': 12,
 '江苏': 13,
 '广东': 14,
 '上海': 15,
 '四川': 16,
 '山西': 17,
 '浙江': 18}

In [14]:
all_games

,轮次,日期,主队,比分,客队,主队比分,客队比分,负方,胜方
0,第3轮,2020-10-22 04:30:00,浙江,105:93,山西,105,93,山西,浙江
1,第4轮,2020-10-24 12:00:00,上海,99:102,北京,99,102,上海,北京
2,第5轮,2020-10-28 03:00:00,青岛,113:114,江苏,113,114,青岛,江苏
3,第7轮,2020-11-01 03:00:00,广州,110:111,同曦,110,111,广州,同曦
4,第8轮,2020-11-04 11:35:00,吉林,105:84,青岛,105,84,青岛,吉林
...,...,...,...,...,...,...,...,...,...
268,第21轮,2020-12-21 04:30:00,天津,91:105,新疆,91,105,天津,新疆
269,第21轮,2020-12-21 07:30:00,北控,112:108,吉林,112,108,吉林,北控
270,第22轮,2020-12-24 04:00:00,浙江,94:106,山西,94,106,浙江,山西
271,第23轮,2020-12-26 11:35:00,辽宁,125:110,山西,125,110,山西,辽宁


In [15]:
all_games['负方']=all_games['负方'].map(lambda team_name: hash_t[team_name])
all_games['胜方']=all_games['胜方'].map(lambda team_name: hash_t[team_name])

In [16]:
all_games

,轮次,日期,主队,比分,客队,主队比分,客队比分,负方,胜方
0,第3轮,2020-10-22 04:30:00,浙江,105:93,山西,105,93,17,18
1,第4轮,2020-10-24 12:00:00,上海,99:102,北京,99,102,15,6
2,第5轮,2020-10-28 03:00:00,青岛,113:114,江苏,113,114,11,13
3,第7轮,2020-11-01 03:00:00,广州,110:111,同曦,110,111,4,3
4,第8轮,2020-11-04 11:35:00,吉林,105:84,青岛,105,84,11,5
...,...,...,...,...,...,...,...,...,...
268,第21轮,2020-12-21 04:30:00,天津,91:105,新疆,91,105,9,8
269,第21轮,2020-12-21 07:30:00,北控,112:108,吉林,112,108,5,0
270,第22轮,2020-12-24 04:00:00,浙江,94:106,山西,94,106,18,17
271,第23轮,2020-12-26 11:35:00,辽宁,125:110,山西,125,110,17,7


In [17]:
all_games.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 273 entries, 0 to 272
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   轮次      273 non-null    object        
 1   日期      273 non-null    datetime64[ns]
 2   主队      273 non-null    object        
 3   比分      273 non-null    object        
 4   客队      273 non-null    object        
 5   主队比分    273 non-null    int64         
 6   客队比分    273 non-null    int64         
 7   负方      273 non-null    int64         
 8   胜方      273 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(4)
memory usage: 31.3+ KB


# Convert to Matrix
1. 273 games as of today (1/16/2021 20:50)
2. 19 teams
3. so there will be 273 equations

In [18]:
matrix=np.zeros([len(all_games),len(hash_t)])
print(matrix.shape)
matrix

(273, 19)


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
for matrix_row,(df_index,df_row) in zip(matrix,all_games.iterrows()):
    matrix_row[df_row['胜方']]=1
    matrix_row[df_row['负方']]=-1
#     print(df_row['胜方'])
#     print(df_row['负方'])
#     print(y[1])

In [20]:
matrix

array([[ 0.,  0.,  0., ...,  0., -1.,  1.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  1., -1.],
       [ 0.,  0.,  0., ...,  0., -1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [21]:
# confirm matrix is correct
for matrix_row,(df_index,df_row) in zip(matrix,all_games.iterrows()):
    if matrix_row.argmax()!=df_row['胜方']:
        print(df_row)
    if matrix_row.argmin()!=df_row['负方']:
        print(df_row)

In [22]:
MOV = abs(all_games['主队比分']-all_games['客队比分'])
MOV

0      12
1       3
2       1
3       1
4      21
       ..
268    14
269     4
270    12
271    15
272    19
Length: 273, dtype: int64

# Construct Model

In [23]:
X_HA = matrix
y=MOV

In [24]:
lin_reg_HA = LinearRegression(fit_intercept=False)
lin_reg_HA.fit(X_HA, y)

LinearRegression(fit_intercept=False)

In [25]:
print(lin_reg_HA.intercept_)
print(lin_reg_HA.coef_)

0.0
[ -2.8176996    0.23965546 -10.59036325 -13.3297517    0.50046568
   0.43318931  -0.25014555  13.57525471   3.29790267  -8.51669881
   1.33046766   2.03168839   4.539866    -7.97120623  12.94034017
   0.08600006  -4.00749116  -1.74874019  10.2572664 ]


In [32]:
rating=pd.DataFrame(list(zip(hash_t.keys(),lin_reg_HA.coef_)),columns=['team','rating'])
rating.sort_values(by='rating',ascending=False)

,team,rating
7,辽宁,13.575255
14,广东,12.940340
18,浙江,10.257266
12,山东,4.539866
8,新疆,3.297903
11,青岛,2.031688
10,广厦,1.330468
4,广州,0.500466
5,吉林,0.433189
1,深圳,0.239655
